## Imports

In [37]:
from prettytable import *
from utils import *
from utils import *
from probabilityPlus import *
from itertools import *

## Função cond_indep

### Explicação

cond_indep começa por chamar a função todosCaminhos que devolva todos os caminhos possiveis entre x e y e iniciliza um inativosCaminho que vai guardar em lista em todos os caminhos se há triplos inativos. Percorre um for por cada caminho e cria os triplos do caminho, depois percorre dois fors, um para cada membro da lista e e por cada triplo e se algum membro de e estiver no triplo quer dizer que é inativo. No fim percorre-se a lista inativosCaminho e se houver algum caminho ativo quer dizer que é condicionalmente independente

In [38]:
def cond_indep(x,y,e,rede):   
    dic=graphToDic(rede)
    caminhos=find_all_paths(dic,x,y,[])
    inativosCaminho=[]
    for i in caminhos:
        temInativos=False
        if i:
            triplos=triplosEmCaminho(i)
        for j in e:
            for t in triplos:
                if j in t:
                    temInativos=True
        inativosCaminho.append(temInativos)
    for i in inativosCaminho:
        if i==False:
            return False
    return True

## Função cond_indep_explica

### Explicação

Começa por chamar a função todosCaminhos que devolve todos os caminhos, ainda inicia duas variáveis, uma para a resposta e outra para os triplos. Depois percorre todos os caminhos e para cada divide em triplos e adiciona à lista triplos. Depois por cada membro da lista triplos chama a função auxiliar checkTriplo que para cada um verifica se é cadeia causal etc... e se é ou não ativo e devolve uma string que é adicionado à lista resposta. Depois percorre a resposta e passa-a para uma string explicacao, depois chama cond_indep e dependendo se é verdadeiro ou falso, adiciona à explicacao e por fim devolve um tuplo com a resposta de cond_indep e com a explicacao

In [70]:
def cond_indep_explica(x,y,e,rede):   
    dic=graphToDic(rede)
    caminhos=find_all_paths(dic,x,y,[])
    triplos=[]
    resposta=[]
    for i in range(0,len(caminhos)):
        triplos=triplosEmCaminho(caminhos[i])
        for triplo in triplos:
            resposta.append((i,checkTriplo(triplo,e,rede)))
    explicacao=""
    for i in range(0,len(caminhos)):
        inicio=0
        explicacao=explicacao+ 'No '+str(i+1)+'º caminho, o '
        for j in range(0,len(resposta)):
            if resposta[j][0]==i:
                inicio=inicio+1
                explicacao=explicacao+str(inicio)+'º triplo é '+resposta[j][1]+', '
    cond=cond_indep(x,y,e,rede)
    if cond:
        explicacao+='logo é condicionalmente independente, porque todos os caminhos são inativos'
    else:       
        explicacao+='logo não é condicionalmente independente, porque tem pelo menos um caminho ativo'
    return (cond,explicacao)

## Função todos_cond_indeps

### Explicação

todos_cond_indeps começa por passar a rede para uma lista de nodes, percorre essa lista três vezes subtraindo o valor escolhido anteriormente e com os três valores dos fors x,y,z chama cond_indep e se o retorno for verdadeiro adiciona o triplo à lista para retornar

In [71]:
def todos_cond_indeps(rede):
    var=set(rede.variables)
    lista=[]
    for x in var:
        otherX=var-{x}
        for y in otherX:
            otherY=otherX-{y}
            for z in otherY:
                i=cond_indep(x,y,[z],rede)
                if i:
                    lista.append([x,y,z])
    return lista

## Funções Auxiliares

In [72]:
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if not start in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

def graphToDic(rede):
    var=set(rede.variables)
    myDict={}
    for v in var:
        myDict[v]=rede.variable_node(v).parents+rede.variable_node(v).children
    return myDict
        
def triplosEmCaminho(caminho):
    triplos=[]
    for i in range(0,len(caminho)):
        if i+2>len(caminho)-1:
            return triplos
        triplos.append((caminho[i],caminho[i+1],caminho[i+2]))
        
def checkTriplo(triplo,e,rede):
    pais1=rede.variable_node(triplo[0]).parents
    pais2=rede.variable_node(triplo[1]).parents
    pais3=rede.variable_node(triplo[2]).parents
    explicacao=""

    if triplo[0] in pais2 and triplo[1] in pais3:
        explicacao += "uma cadeia causal "
    if triplo[0] in pais2 and triplo[2] in pais2:
        explicacao += "um efeito comum "
    if triplo[1] in pais1 and triplo[1] in pais3:
        explicacao += "uma causa comum "
    for i in e:
        if i in triplo:
            return explicacao + "inativo"
    return explicacao + "ativo"
    

### todosCaminhos

Recebendo um node x e node y e uma rede retorna todos os caminhos possíveis de x a y na rede, utilizando uma função auxiliar caminhosAux

### triplosEmCaminho

Recebendo um caminho que é uma lista de nodes retorna uma lista com os triplos do caminho


### CheckTriplo

Recebendo um triplo, uma lista de nodes e uma rede verifica de que tipo é o triplo e se é ativo ou inativo dado a lista de nodes

## Testes

Vamos testar para o caso da rede de Bayes Burglary.

In [73]:
T, F = True, False
burglary = BayesNet([
    ('Burglary', '', 0.001),
    ('Earthquake', '', 0.002),
    ('Alarm', 'Burglary Earthquake',
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('JohnCalls', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryCalls', 'Alarm', {T: 0.70, F: 0.01})
    ])

Agora vamos testar para ver se Burglary e Alarm são condicionalmente independentes dado Earthquake...

In [74]:
X = 'Burglary'
Y = 'Alarm'
E = 'Earthquake'

print(cond_indep_explica(X, Y, E, burglary))

(False, 'No 1º caminho, o logo não é condicionalmente independente, porque tem pelo menos um caminho ativo')


Agora vamos ver se JohnCalls é condicionalmente independente de Alarm dado Earthquake...

In [75]:
X = 'JohnCalls'
Y = 'Alarm'
E = 'Earthquake'

print(cond_indep_explica(X, Y, E, burglary))

(False, 'No 1º caminho, o logo não é condicionalmente independente, porque tem pelo menos um caminho ativo')


### Caso do exemplo no enunciado

In [76]:
grafo=BayesNet([
    ('A', '', 0.5),
    ('B', 'A', {T: 0.90, F: 0.05}),
    ('D', 'A', {T: 0.90, F: 0.05}),
    ('C', 'B', {T: 0.90, F: 0.05}),
    ('E', 'B D', {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ])

print(grafo)

R = grafo

print("Aceder ao no na posicao 2:")
print(R.nodes[2])

print("\nAceder ao nome do no na posicao 2:")
print(R.variables[2])

print("\nAceder ao numero de nos da rede")
print(len(R.nodes))

print("\nAceder aos nomes dos filhos da posicao 2:")
print(R.nodes[2].children)


BayesNet([('A', ''), ('B', 'A'), ('D', 'A'), ('C', 'B'), ('E', 'B D')])
Aceder ao no na posicao 2:
('D', 'A')

Aceder ao nome do no na posicao 2:
D

Aceder ao numero de nos da rede
5

Aceder aos nomes dos filhos da posicao 2:
['E']


Vamos ver se A e C são condicionalmente independente dado B

In [77]:
print(cond_indep('A', 'C', 'B', grafo))

True


Vamos ver se A e C são condicionalmente independente dado E

In [78]:
print(cond_indep('A', 'C', 'E', grafo))

False


Vamos então agora explicar os dois exemplos

In [79]:
print(cond_indep_explica('A', 'C', 'B', grafo))
print(cond_indep_explica('A', 'C', 'E', grafo))

(True, 'No 1º caminho, o 1º triplo é uma cadeia causal inativo, No 2º caminho, o 1º triplo é uma cadeia causal ativo, 2º triplo é um efeito comum inativo, 3º triplo é uma causa comum inativo, logo é condicionalmente independente, porque todos os caminhos são inativos')
(False, 'No 1º caminho, o 1º triplo é uma cadeia causal ativo, No 2º caminho, o 1º triplo é uma cadeia causal inativo, 2º triplo é um efeito comum inativo, 3º triplo é uma causa comum inativo, logo não é condicionalmente independente, porque tem pelo menos um caminho ativo')


Agora vamos ver todos os triplos distintos  (𝑋,𝑌,𝐸)  tais que se verifica que  𝑋  é condicionalmente independente de  𝑌  dado  𝐸  na rede  𝑅 .

In [49]:
print(todos_cond_indeps(grafo))

[['A', 'C', 'B'], ['D', 'C', 'B'], ['E', 'C', 'B'], ['C', 'D', 'B'], ['C', 'A', 'B'], ['C', 'E', 'B']]


# IMPORTANTE - a não esquecer

Um triplo (No1, No2, No3) é considerado como sendo do tipo cadeia-causal sempre que, na rede em causa, os 3 nós estão ligados por arcos com a mesma direção.
Isto é, tanto podemos ter No1 -> No2 ->No3 como No1 <- No2 <- No3.


Na verdade, para determinarmos o tipo de um triplo só precisamos de olhar  para a direção dos seus dois arcos.
Por isso só há mesmo 3 tipos de triplos:
- cadeia-causal: arcos na mesma direção 
- causa-comum: arcos em direções opostas, divergindo a partir do nó do meio 
- efeito-comum: arcos em direções opostas, convergindo para o nó do meio

Portanto, qualquer que seja o triplo (No1,No2,No3), o triplo inverso (No3,No2,No1) é sempre do mesmo tipo.

### ---------------------

O conjunto de nós E pode ser representado em Python de diferentes formas:
- uma lista sem elementos repetidos (em que a ordem dos elementos não é relevante)
- um conjunto
- Uma string contendo os nomes das variáveis separadas por espaços

Podem escolher a representação interna que quiserem e desenvolver a função tendo em conta essa representação. 
Idealmente poderão depois estender a função de modo a que aceite esse parâmetro dado sob qualquer dessas formas (tal como, por exemplo em probabilityPlus.py o construtor de um BayesNode permite que os pais de um nó sejam indicados de diferentes formas).

A representação escolhida/permitida para os parâmetros deve ser indicada no relatório.

### ---------------------

Será que tem de ser acrescentar na funcao cond_indep_explica quando Por exemplo:

- X é descendente de Y & Y é descendente de X & E contem X ????

Perguntar aos docentes ????